# Definition du dataset Toy_dataset

In [1]:
toy_dataset = [
    ['Skirt', 'Sneakers', 'Scarf', 'Pants', 'Hat'],
    ['Sunglasses', 'Skirt', 'Sneakers', 'Pants', 'Hat'],
    ['Dress', 'Sandals', 'Scarf', 'Pants', 'Heels'],
    ['Dress', 'Necklace', 'Earrings', 'Scarf', 'Hat', 'Heels', 'Hat'],
    ['Earrings', 'Skirt', 'Skirt', 'Scarf', 'Shirt', 'Pants']
]


# Encodage des transactions

In [2]:
import pandas as pd
from mlxtend.preprocessing import TransactionEncoder
from mlxtend.frequent_patterns import apriori, association_rules

# Toy dataset
toy_dataset = [['Skirt', 'Sneakers', 'Scarf', 'Pants', 'Hat'],
               ['Sunglasses', 'Skirt', 'Sneakers', 'Pants', 'Hat'],
               ['Dress', 'Sandals', 'Scarf', 'Pants', 'Heels'],
               ['Dress', 'Necklace', 'Earrings', 'Scarf', 'Hat', 'Heels', 'Hat'],
               ['Earrings', 'Skirt', 'Skirt', 'Scarf', 'Shirt', 'Pants']]

# Transformer en one-hot encoding
te = TransactionEncoder()
te_ary = te.fit(toy_dataset).transform(toy_dataset)
df = pd.DataFrame(te_ary, columns=te.columns_)

# Appliquer apriori avec un support minimum (ex: 0.6)
frequent_itemsets = apriori(df, min_support=0.6, use_colnames=True)

print("Itemsets fréquents:")
print(frequent_itemsets)

# Générer les règles avec confiance >= 0.7
rules = association_rules(frequent_itemsets, metric="confidence", min_threshold=0.7)

print("\nRègles d'association:")
print(rules[['antecedents', 'consequents', 'support', 'confidence', 'lift']])


Itemsets fréquents:
   support        itemsets
0      0.6           (Hat)
1      0.8         (Pants)
2      0.8         (Scarf)
3      0.6         (Skirt)
4      0.6  (Scarf, Pants)
5      0.6  (Pants, Skirt)

Règles d'association:
  antecedents consequents  support  confidence    lift
0     (Scarf)     (Pants)      0.6        0.75  0.9375
1     (Pants)     (Scarf)      0.6        0.75  0.9375
2     (Pants)     (Skirt)      0.6        0.75  1.2500
3     (Skirt)     (Pants)      0.6        1.00  1.2500


(Scarf) ⇒ (Pants)	0.60	0.75	0.94 , 75% des transactions contenant une écharpe contiennent aussi un pantalon.
 ! Mais le lift < 1 signifie que cette association est plus faible que le hasard, donc l’achat des deux ensemble n’est pas particulièrement significatif.
Pants) ⇒ (Scarf)	0.60	0.75	0.94 75% des transactions contenant un pantalon contiennent aussi une écharpe.
 ! Lift < 1 également, donc même analyse : association faible malgré la confiance.
(Pants) ⇒ (Skirt)	0.60	0.75	1.25 , 75% des acheteurs de pantalon achètent aussi une jupe.
 ! Le lift > 1 montre une association positive, les deux produits sont acheté ensemble plus souvent que par hasard.
(Skirt) ⇒ (Pants)	0.60	1.00	1.25 , 100% des acheteurs de jupe achètent aussi un pantalon.
 Confiance maximale (1.0) + lift > 1 : c’est une très forte règle d’association. Cela peut justifier une offre groupée ou un bundle.

# Exploration de l'ensemble des données du checkpoint 

In [12]:
#  Étape 1 : Importations
import pandas as pd
from mlxtend.preprocessing import TransactionEncoder
from mlxtend.frequent_patterns import apriori, association_rules

#  Étape 2 : Chargement et transformation des données
df = pd.read_csv("Market_Basket_Optimisation.csv", header=None)

# Création de la liste des transactions
transactions = []
for i in range(df.shape[0]):
    transactions.append([str(item).strip() for item in df.iloc[i] if pd.notna(item)])

# Étape 3 : Encodage avec TransactionEncoder
te = TransactionEncoder()
te_ary = te.fit(transactions).transform(transactions)
df_encoded = pd.DataFrame(te_ary, columns=te.columns_)

# Étape 4 : Algorithme Apriori
frequent_itemsets = apriori(df_encoded, min_support=0.01, use_colnames=True)

#  Étape 5 : Génération des règles d'association
rules = association_rules(frequent_itemsets, metric="lift", min_threshold=1)

#  Étape 6 : Filtrage et nettoyage pour affichage clair
rules_filtered = rules[
    (rules['lift'] > 2) & 
    (rules['confidence'] > 0.4) & 
    (rules['support'] > 0.01)
].sort_values(by='lift', ascending=False)

# Conversion frozenset → texte simple
rules_filtered['antecedents'] = rules_filtered['antecedents'].apply(lambda x: ', '.join(list(x)))
rules_filtered['consequents'] = rules_filtered['consequents'].apply(lambda x: ', '.join(list(x)))

# Arrondir les métriques
rules_filtered[['support', 'confidence', 'lift']] = rules_filtered[['support', 'confidence', 'lift']].round(3)

#  Étape 7 : Affichage final des règles
rules_display = rules_filtered[['antecedents', 'consequents', 'support', 'confidence', 'lift']]
print(" Règles d'association les plus fortes :\n")
print(rules_display.head(10))


📊 Règles d'association les plus fortes :

                    antecedents    consequents  support  confidence   lift
384  ground beef, mineral water      spaghetti    0.017       0.417  2.395
342           eggs, ground beef  mineral water    0.010       0.507  2.126
376           milk, ground beef  mineral water    0.011       0.503  2.110


1. ground beef + mineral water → spaghetti
Support : 1.7% des paniers.

Confidence : 41.7% des clients qui achètent du bœuf haché et de l’eau minérale achètent aussi des spaghettis.

Lift : 2.395 → Ils achètent spaghetti 2,4 fois plus souvent que par hasard.

 Interprétation :

Ces trois produits semblent liés à la préparation d’un repas complet (plat de pâte + viande + boisson). Cela suggère qu’ils sont souvent achetés ensemble.

 Proposition :

Mettre en avant un "Pack repas familial" : 1kg de spaghetti + 500g de bœuf + 2 bouteilles d’eau à prix réduit.

2. eggs + ground beef → mineral water
Support : 1.0% des paniers.

Confidence : 50.7%

Lift : 2.126

 Interprétation :

Les clients qui prennent des œufs et du bœuf haché prennent très souvent aussi de l’eau minérale. On peut supposer qu’ils préparent un repas ou font leurs courses principales.

 Proposition :

Créer un rayon “repas équilibré” combinant protéines animales + eau minérale. Éventuellement proposer des offres fidélité ciblées.

3. milk + ground beef → mineral water
Support : 1.1%

Confidence : 50.3%

Lift : 2.110

 Interprétation :

Ici encore, l’eau minérale revient dès qu’il y a des produits frais ou de base. Cela peut traduire un comportement d’achat “familial” ou “hebdomadaire”.

 Proposition :

Créer un pack “panier frais” : lait + viande + eau. Ces produits peuvent être regroupés en tête de gondole.

